In [1]:
QRELS_PATH = 'evaluation/qrels.json'

In [33]:
SEED = 2024
# ANALYSSIS_COL = 'userAcceleration.x'
# PRECOMPUTED_DISTANCES = f"evaluation/ddtw/{ANALYSIS_COL.replace('.', '-')}_distances.npy"
# RUN_FILE = f"evaluation/runs/ddtw_{ANALYSIS_COL.replace('.', '-')}.json"

DIST = 'adtw'
PRECOMPUTED_DISTANCES = f"evaluation/multivar/{DIST}_distances.npy"
RUN_FILE = f"evaluation/runs/{DIST}_multivar.json"

# Preparando avaliação

In [34]:
from ranx import Qrels

In [35]:
qrels = Qrels.from_file(QRELS_PATH)

# Executando a busca por similaridade

In [36]:
from ranx import Run
import numpy as np
from aeon.distances import dtw_pairwise_distance

In [37]:
def compute_neighbors_by_similarity(
    distances: np.array,
    topk: int = 100) -> dict:

  run_dict = {}

  for qid in range(distances.shape[0]):
    all_sims = distances[qid]
    topk = np.argsort(all_sims)

    # ignorando o proprio exemplo de busca
    estimated_ids = topk[1:]
    scores = list(range(len(estimated_ids)))[::-1]

    run_dict[str(qid)] = {str(n): float(score) for n, score in zip(estimated_ids, scores)}

  return run_dict

In [38]:
PRECOMPUTED_DISTANCES

'evaluation/multivar/adtw_distances.npy'

In [39]:
distances = np.load(PRECOMPUTED_DISTANCES)

In [40]:
run_dict = compute_neighbors_by_similarity(distances)

# Avaliando métricas de RI

In [41]:
from ranx import evaluate

In [42]:
METRICS = ["recall@10", 'map@10', 'hit_rate@10', 'mrr@10']
# METRICS = 'mrr@10'

In [43]:
run = Run(run_dict)

In [44]:
metrics = evaluate(qrels, run, METRICS)
metrics

{'recall@10': 0.24074074074074078,
 'map@10': 0.19646795543062906,
 'hit_rate@10': 0.9652777777777778,
 'mrr@10': 0.8069527116402117}

# Serializando Run

In [45]:
RUN_FILE

'evaluation/runs/adtw_multivar.json'

In [46]:
run.save(RUN_FILE)